1- Importar bibliotecas

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import * #StructType, StructField, StringType
from pyspark.sql import functions as F
from datetime import datetime
import pandas as pd
from pandas import DataFrame
import os

2- Ler arquivo .csv

In [0]:
%python
file_path = "/FileStore/tables/dados_bancarios.csv"

df_bank = spark.read.csv(file_path)
display(df_bank)

_c0
Número da Conta;Tipo de Conta;Saldo;Moeda;Data da Última Movimentação
1;Corrente;8649.73;BRL;27/11/2024
2;Corrente;4307.05;BRL;10/06/2024
3;Corrente;8321.04;BRL;04/05/2024
4;Corrente;1474.27;BRL;18/08/2024
5;Corrente;4028.64;BRL;08/08/2024
6;Poupança;4964.52;BRL;05/02/2025
7;Poupança;733.73;BRL;13/06/2024
8;Corrente;8903.7;BRL;28/06/2024
9;Poupança;655.1;BRL;02/09/2024


3- Estruturar o arquivo .csv definindo as colunas do df (nome, tipo, se pode ou não nulo)

In [0]:
%python
file_schema = StructType([
    StructField(('Número da Conta'), IntegerType(), False)
    ,StructField(('Tipo de Conta'), StringType(), False)
    ,StructField(('Saldo'), FloatType(), False)
    ,StructField(('Moeda'), StringType(), False)
    ,StructField(('Data da Última Movimentação'), DateType(), False)
])

df_bank = spark.read.format("csv")\
    .option("inferSchema", "true") \
    .option("schema", file_schema) \
    .option("header", "true") \
    .option("sep", ";") \
    .option("encoding", "utf-8") \
    .load(file_path)

df_bank.show()

+---------------+-------------+-------+-----+---------------------------+
|Número da Conta|Tipo de Conta|  Saldo|Moeda|Data da Última Movimentação|
+---------------+-------------+-------+-----+---------------------------+
|              1|     Corrente|8649.73|  BRL|                 2024-11-27|
|              2|     Corrente|4307.05|  BRL|                 2024-06-10|
|              3|     Corrente|8321.04|  BRL|                 2024-05-04|
|              4|     Corrente|1474.27|  BRL|                 2024-08-18|
|              5|     Corrente|4028.64|  BRL|                 2024-08-08|
|              6|     Poupança|4964.52|  BRL|                 2025-02-05|
|              7|     Poupança| 733.73|  BRL|                 2024-06-13|
|              8|     Corrente| 8903.7|  BRL|                 2024-06-28|
|              9|     Poupança|  655.1|  BRL|                 2024-09-02|
|             10|     Corrente|9430.55|  BRL|                 2024-08-03|
|             11|     Corrente|8293.79

In [0]:
# Visualizar como está a estrutura de cada coluna
df_bank.printSchema()

root
 |-- Número da Conta: integer (nullable = true)
 |-- Tipo de Conta: string (nullable = true)
 |-- Saldo: double (nullable = true)
 |-- Moeda: string (nullable = true)
 |-- Data da Última Movimentação: date (nullable = true)



In [0]:
# Quantidade de linhas no dataframe
number_rows = df_bank.count()

# Quantidade de colunas no dataframe
# dtypes = info
number_columns = len(df_bank.dtypes)

print(f'Number of rows: {number_rows}')
print(f'Row: {df_bank.head(1)}\n')
print(f'Number of columns: {number_columns}')
print(f'Columns: {df_bank.columns}')

Number of rows: 1001
Row: [Row(Número da Conta=1, Tipo de Conta='Corrente', Saldo=8649.73, Moeda='BRL', Data da Última Movimentação=datetime.date(2024, 11, 27))]

Number of columns: 5
Columns: ['Número da Conta', 'Tipo de Conta', 'Saldo', 'Moeda', 'Data da Última Movimentação']


4- Filtros e agregações no dataframe

In [0]:
# Saldo total de cada tipo de conta
agregacao_saldo_bank = df_bank.groupBy(F.col('Tipo de Conta')).agg(round(F.sum(F.col('Saldo')),2).alias('Saldo_Total'))
agregacao_saldo_bank.show() 

+-------------+-----------+
|Tipo de Conta|Saldo_Total|
+-------------+-----------+
|     Corrente| 2676887.22|
|     Poupança| 2484726.01|
+-------------+-----------+



In [0]:
# Quantidade de contas com a última movimentação entre os meses de janeiro e abril
filtrar_bank_mes = df_bank.filter(F.month(F.col('Data da Última Movimentação')).between(1,4))
filtrar_bank_mes.show()

+---------------+-------------+-------+-----+---------------------------+
|Número da Conta|Tipo de Conta|  Saldo|Moeda|Data da Última Movimentação|
+---------------+-------------+-------+-----+---------------------------+
|              6|     Poupança|4964.52|  BRL|                 2025-02-05|
|             13|     Corrente|8469.22|  BRL|                 2025-02-25|
|             14|     Poupança|4490.05|  BRL|                 2025-01-10|
|             21|     Poupança|8365.89|  BRL|                 2024-04-08|
|             22|     Poupança|7236.73|  BRL|                 2024-04-22|
|             26|     Poupança|3842.04|  BRL|                 2025-03-13|
|             31|     Poupança|7089.74|  BRL|                 2024-04-18|
|             32|     Poupança| 1883.9|  BRL|                 2025-01-05|
|             35|     Poupança|1685.81|  BRL|                 2024-04-29|
|             37|     Poupança|3868.67|  BRL|                 2024-04-03|
|             38|     Poupança|3864.78

5- Salvar em formato parquet

In [0]:
df_bank.write.parquet('/path/to/save/df_file_bank.parquet')

6- Visualizar o dataframe em uma view (SQL)

In [0]:
df_bank.createOrReplaceTempView("dados_bancarios")

In [0]:
%sql
-- Visualizar a view
SELECT * FROM dados_bancarios

Número da Conta,Tipo de Conta,Saldo,Moeda,Data da Última Movimentação
1,Corrente,8649.73,BRL,2024-11-27
2,Corrente,4307.05,BRL,2024-06-10
3,Corrente,8321.04,BRL,2024-05-04
4,Corrente,1474.27,BRL,2024-08-18
5,Corrente,4028.64,BRL,2024-08-08
6,Poupança,4964.52,BRL,2025-02-05
7,Poupança,733.73,BRL,2024-06-13
8,Corrente,8903.7,BRL,2024-06-28
9,Poupança,655.1,BRL,2024-09-02
10,Corrente,9430.55,BRL,2024-08-03


7- Fazendo agregações simples utilizando a view: dados_bancarios

In [0]:
%sql
-- Saldo total de cada tipo de conta
SELECT `Tipo de Conta` as Tipo_Conta
,ROUND(SUM(Saldo),2) as Saldo_Total
FROM dados_bancarios
GROUP BY `Tipo de Conta`

Tipo_Conta,Saldo_Total
Corrente,2676887.22
Poupança,2484726.01


In [0]:
%sql
-- Quantidade de contas com a última movimentação entre os meses de janeiro e abril
SELECT COUNT(`Número da Conta`) as Qtde_Conta
FROM dados_bancarios
WHERE MONTH(`Data da Última Movimentação`) BETWEEN 01 AND 04

Qtde_Conta
340
